### Import Library

In [1]:
import pandas as pd
import numpy as np
import os

# string processing
import re
from keras.utils.np_utils import to_categorical
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing, feature_selection, metrics


### Import true label

In [2]:
# import the true label

df_merge_quality = pd.read_csv('data/US_patent_abstract_5000_2015_with_title_1_5y.csv')

In [3]:
y_test_true = df_merge_quality['quality_rank'][4000:].values
print(y_test_true.shape)
y_test_true[:10]

(1000,)


array([0, 1, 0, 0, 0, 1, 1, 1, 1, 1])

In [4]:
# double check the label distribution
# Validation set

count = len(y_test_true)
positive = sum(y_test_true)
negative = sum(x == 0 for x in y_test_true)
    
print(f'Total Sample Count = {count} \nPositive Label Count = {positive} \nNegative Label Count = {negative} \nPositive Class Ratio = {positive/count}  \nNegative Class Ratio = {negative/count}')


Total Sample Count = 1000 
Positive Label Count = 405 
Negative Label Count = 595 
Positive Class Ratio = 0.405  
Negative Class Ratio = 0.595


### Create Majority Class Prediction file as one baseline to add into the pool

In [5]:
mc_class = np.zeros(1000)
mc_class
mc_class.shape

(1000,)

In [6]:
np.savetxt("Prediction_Output/Majority_Class_prediction.csv", mc_class)

### Import the prediction output files
#### Let's first look at couple example of the prediction output file

In [7]:
# Option 1: Use .read.csv
test_read1 = pd.read_csv("Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv", 
                        names = ['Class 0', 'Class 1'], 
                        sep=" ")
test_read1

,Class 0,Class 1
0,0.847997,0.130727
1,0.332160,0.667886
2,0.236171,0.788574
3,0.887799,0.081256
4,0.828767,0.121260
...,...,...
995,0.540695,0.398545
996,0.932154,0.059405
997,0.195182,0.780047
998,0.534664,0.487115


In [8]:
# Option 2: Use .readlines
temp_pred = []
with open("Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv") as f1:
    for line in f1.readlines():
        print([float(x) for x in line.split()])

[0.8479969501495361, 0.13072672486305237]
[0.3321598172187805, 0.6678864359855652]
[0.2361709177494049, 0.7885740995407104]
[0.8877992033958435, 0.08125635981559753]
[0.8287674188613892, 0.12126028537750244]
[0.6365858316421509, 0.30468297004699707]
[0.7011922597885132, 0.24463146924972534]
[0.8038750290870667, 0.13103100657463074]
[0.8003423810005188, 0.19075867533683777]
[0.9636895656585693, 0.02832663059234619]
[0.8826712965965271, 0.09366616606712341]
[0.738400399684906, 0.2574821710586548]
[0.7997338175773621, 0.19859188795089722]
[0.9297205209732056, 0.05143982172012329]
[0.788860023021698, 0.21694540977478027]
[0.8776333332061768, 0.13374078273773193]
[0.8845688104629517, 0.0818154513835907]
[0.8796502351760864, 0.11024996638298035]
[0.9105886220932007, 0.10028195381164551]
[0.7579448223114014, 0.25525808334350586]
[0.35311049222946167, 0.6041271090507507]
[0.8072425723075867, 0.19061025977134705]
[0.5108556747436523, 0.4793073534965515]
[0.41507914662361145, 0.6663478016853333]

[0.6374702453613281, 0.3567034602165222]
[0.9068436622619629, 0.0944732129573822]
[0.8995609283447266, 0.10058152675628662]
[0.6534614562988281, 0.32492518424987793]
[0.866028368473053, 0.134196937084198]
[0.3428957164287567, 0.6350319385528564]
[0.9377009868621826, 0.044043391942977905]
[0.669268786907196, 0.36409151554107666]
[0.8982534408569336, 0.09441012144088745]
[0.7992210388183594, 0.16347354650497437]
[0.7178131341934204, 0.2597239017486572]
[0.8986403942108154, 0.06655105948448181]
[0.4523294270038605, 0.5395923256874084]
[0.7010652422904968, 0.3322826325893402]
[0.5248283743858337, 0.41858744621276855]
[0.915253758430481, 0.06817862391471863]
[0.43677496910095215, 0.48884502053260803]
[0.4695812463760376, 0.4888046979904175]
[0.7300695180892944, 0.2971082329750061]
[0.26449859142303467, 0.7306581139564514]
[0.5215172171592712, 0.5221426486968994]
[0.7844823598861694, 0.16705858707427979]
[0.5521592497825623, 0.4908233880996704]
[0.4724537432193756, 0.4565327763557434]
[0.774

In [9]:
# We can get the predicted class this way
temp_pred2 = []
with open("Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv") as f1:
    for line in f1.readlines():
        temp_pred2.append(np.argmax([float(x) for x in line.split()]))
temp_pred2

[0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,


In [10]:
# We can get the predicted class this way
temp_pred2 = []
with open('Prediction_Output/BERT_CNN_5yr_claims_last_510_words_dev_prob.csv') as f1:
    for line in f1.readlines():
        temp_pred2.append(np.argmax([float(x) for x in line.split()]))
sum(temp_pred2)


33

### Create a list holding all filenames to generate combinations pair later

In [11]:
# https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory

import os

directory = "Prediction_Output"

file_name = []

for filename in os.listdir(directory):
    if filename.endswith(".csv") and not('50k' in filename): # not include 50k version for now
        file_name.append(os.path.join(directory, filename))

file_name

['Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
 'Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv',
 'Prediction_Output/HATT.csv',
 'Prediction_Output/Distil-BERT_5yr_claims_dev_prob-Copy1.csv',
 'Prediction_Output/BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv',
 'Prediction_Output/Majority_Class_prediction.csv',
 'Prediction_Output/CNN_5yr_claims_dev_prob-Copy1.csv',
 'Prediction_Output/BERT_CNN_5yr_claims_last_510_words_dev_prob.csv',
 'Prediction_Output/BERT_fine_tune_5yr_abstract_title_dev_prob-Copy1.csv',
 'Prediction_Output/BERT_CNN_5yr_claims_dev_prob-Copy1.csv',
 'Prediction_Output/HATT_abst.csv',
 'Prediction_Output/BERT_fine_tune_5yr_claims_dev_prob_last_n_words.csv',
 'Prediction_Output/LSTM.csv',
 'Prediction_Output/TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv',
 'Prediction_Output/CNN_5yr_abstract_title_dev_prob-Copy1.csv',
 'Prediction_Output/LSTM_abst.csv']

In [12]:
# We can get the predicted class this way
temp_pred2 = []
with open('Prediction_Output/BERT_CNN_5yr_claims_last_510_words_dev_prob.csv') as f1:
    for line in f1.readlines():
        temp_pred2.append(np.argmax([float(x) for x in line.split()]))
sum(temp_pred2)


33

In [13]:
# We can get the predicted class this way
temp_pred2 = []
with open('Prediction_Output/BERT_CNN_5yr_claims_dev_prob-Copy1.csv') as f1:
    for line in f1.readlines():
        temp_pred2.append(np.argmax([float(x) for x in line.split()]))
sum(temp_pred2)


325

### Side-Task: For summary purpose, show the metrics for all files

In [16]:
# define a helper function for below to clean up filename
def clean_filename(filepath):
    '''
    filepah: to pass in string
    '''
    
    if '/' in filepath: # to show only the filename not the path
        file_name = filepath.split('/')[-1]
    else:
        file_name = filepath
    
    return file_name

In [20]:
summary_df = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision', 'Recall','F1','AUC'])

for file in file_name:
    temp_pred_result = []
    with open(file) as f1:
        for line in f1.readlines():
            temp_pred_result.append(np.argmax([float(x) for x in line.split()]))
    #print(file)
    
    accuracy = metrics.accuracy_score(y_test_true, temp_pred_result)
    precision = metrics.precision_score(y_test_true, temp_pred_result)
    recall = metrics.recall_score(y_test_true, temp_pred_result)
    f1 = metrics.f1_score(y_test_true, temp_pred_result)
    auc = metrics.roc_auc_score(y_test_true, temp_pred_result)

    row_to_add = {'Model': clean_filename(file), 'Accuracy': round(accuracy,3), 'Precision':round(precision,3), 'Recall': round(recall,3) ,'F1': round(f1,3) ,'AUC': round(auc, 3) }
    
    summary_df = summary_df.append(row_to_add, ignore_index = True)
    

/home/lin_menghsien/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
summary_df

,Model,Accuracy,Precision,Recall,F1,AUC
0,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,0.600,0.532,0.104,0.174,0.521
1,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.603,0.531,0.170,0.258,0.534
2,HATT.csv,0.699,0.693,0.462,0.554,0.661
3,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.627,0.603,0.232,0.335,0.564
4,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.628,0.577,0.306,0.400,0.577
5,Majority_Class_prediction.csv,0.595,0.000,0.000,0.000,0.500
6,CNN_5yr_claims_dev_prob-Copy1.csv,0.625,0.549,0.417,0.474,0.592
7,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,0.606,0.667,0.054,0.100,0.518
8,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,0.616,0.538,0.363,0.434,0.576
9,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,0.604,0.514,0.412,0.458,0.573


In [22]:
print("Rank by Accuracy")
summary_df.sort_values(by='Accuracy', ascending = False)


Rank by Accuracy


,Model,Accuracy,Precision,Recall,F1,AUC
2,HATT.csv,0.699,0.693,0.462,0.554,0.661
11,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.689,0.636,0.543,0.586,0.666
12,LSTM.csv,0.681,0.623,0.538,0.577,0.658
4,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.628,0.577,0.306,0.400,0.577
3,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.627,0.603,0.232,0.335,0.564
6,CNN_5yr_claims_dev_prob-Copy1.csv,0.625,0.549,0.417,0.474,0.592
13,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.618,0.632,0.136,0.224,0.541
8,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,0.616,0.538,0.363,0.434,0.576
7,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,0.606,0.667,0.054,0.100,0.518
9,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,0.604,0.514,0.412,0.458,0.573


In [23]:
print("Rank by Precision")
summary_df.sort_values(by='Precision', ascending = False)


Rank by Precision


,Model,Accuracy,Precision,Recall,F1,AUC
2,HATT.csv,0.699,0.693,0.462,0.554,0.661
7,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,0.606,0.667,0.054,0.100,0.518
11,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.689,0.636,0.543,0.586,0.666
13,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.618,0.632,0.136,0.224,0.541
12,LSTM.csv,0.681,0.623,0.538,0.577,0.658
3,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.627,0.603,0.232,0.335,0.564
4,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.628,0.577,0.306,0.400,0.577
6,CNN_5yr_claims_dev_prob-Copy1.csv,0.625,0.549,0.417,0.474,0.592
8,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,0.616,0.538,0.363,0.434,0.576
0,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,0.600,0.532,0.104,0.174,0.521


### Ensemble Strategy: 
The strategy of ensemble is to average the predicted probability for both Class 0 and Class 1 from  two different models, then take argmax to determine the class by the larger average probability.

### Start with 2 models combination:

In [45]:

# To generate all combinatition of pairs of models
# https://www.geeksforgeeks.org/itertools-combinations-module-python-print-possible-combinations/
from itertools import combinations

pair_iterator_2 = list(combinations(file_name, 2))

In [46]:
len(pair_iterator_2)

120

In [47]:
pair_iterator_2

[('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/HATT.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/Distil-BERT_5yr_claims_dev_prob-Copy1.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/Majority_Class_prediction.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/CNN_5yr_claims_dev_prob-Copy1.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/BERT_CNN_5yr_claims_last_510_words_dev_prob.csv'),
 ('Prediction_Output/TFIDF_Naiv

In [51]:
# To show all rows (optional)
pd.set_option('display.max_rows', 200)
#pd.reset_option('display.max_rows') # back to default

In [49]:
# for this purpose, since I need to average two files together, I feel it's easier to use pandas dataframe and use vectorized operation

df_ensemble_compare_2 = pd.DataFrame(columns = ['Model 1', 'Model 2', 'Accuracy', 'Precision', 'Recall','F1','AUC'])

for files in pair_iterator_2:
    temp_pred_result = []
    
    
    # temporary hold of data from two files
    df_file_1 = pd.read_csv(files[0], 
                            names = ['Class 0', 'Class 1'], 
                            sep=" ")

    df_file_2 = pd.read_csv(files[1], 
                            names = ['Class 0', 'Class 1'], 
                            sep=" ")
    
    # average two files
    #df_combine = df_file_1 * 0.5 + df_file_2 * 0.5
    df_combine = (df_file_1 + df_file_2) / 2

    # compute the new predicted output
    for i in df_combine.itertuples():
        temp_pred_result.append(np.argmax([i._1, i._2]))
    temp_pred_result
    
    accuracy = metrics.accuracy_score(y_test_true, temp_pred_result)
    precision = metrics.precision_score(y_test_true, temp_pred_result)
    recall = metrics.recall_score(y_test_true, temp_pred_result)
    f1 = metrics.f1_score(y_test_true, temp_pred_result)
    auc = metrics.roc_auc_score(y_test_true, temp_pred_result)
    
    
    row_to_add = {'Model 1': clean_filename(files[0]), 'Model 2': clean_filename(files[1]), 'Accuracy': accuracy, 'Precision':precision, 'Recall': recall,'F1': f1,'AUC': auc}
    
    df_ensemble_compare_2 = df_ensemble_compare_2.append(row_to_add, ignore_index = True)
    
df_ensemble_compare_2

,Model 1,Model 2,Accuracy,Precision,Recall,F1,AUC
0,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.608,0.571429,0.128395,0.209677,0.531424
1,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,0.698,0.784530,0.350617,0.484642,0.642536
2,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.618,0.666667,0.113580,0.194093,0.537462
3,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.628,0.627907,0.200000,0.303371,0.559664
4,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,Majority_Class_prediction.csv,0.405,0.405000,1.000000,0.576512,0.500000
...,...,...,...,...,...,...,...
115,LSTM.csv,CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.646,0.596226,0.390123,0.471642,0.605146
116,LSTM.csv,LSTM_abst.csv,0.670,0.633452,0.439506,0.518950,0.633198
117,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.614,0.546341,0.276543,0.367213,0.560120
118,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,LSTM_abst.csv,0.616,0.551724,0.276543,0.368421,0.561801


In [52]:
print("Rank by Accuracy")
df_ensemble_compare_2.sort_values(by = "Accuracy", ascending = False)

Rank by Accuracy


,Model 1,Model 2,Accuracy,Precision,Recall,F1,AUC
1,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,0.698,0.784530,0.350617,0.484642,0.642536
37,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.697,0.675862,0.483951,0.564029,0.662984
39,HATT.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.691,0.758065,0.348148,0.477157,0.636259
30,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.691,0.695122,0.422222,0.525346,0.648086
38,HATT.csv,LSTM.csv,0.690,0.644377,0.523457,0.577657,0.663409
110,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,LSTM.csv,0.686,0.635821,0.525926,0.575676,0.660442
35,HATT.csv,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,0.683,0.648649,0.474074,0.547789,0.649642
41,HATT.csv,LSTM_abst.csv,0.682,0.652632,0.459259,0.539130,0.646436
29,HATT.csv,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.682,0.693333,0.385185,0.495238,0.634609
15,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,0.681,0.712871,0.355556,0.474465,0.629038


In [53]:
print("Rank by Precision")
df_ensemble_compare_2.sort_values(by = "Precision", ascending = False)

Rank by Precision


,Model 1,Model 2,Accuracy,Precision,Recall,F1,AUC
33,HATT.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,0.669,0.798387,0.244444,0.374291,0.601214
1,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,0.698,0.784530,0.350617,0.484642,0.642536
10,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.668,0.774436,0.254321,0.382900,0.601950
87,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.625,0.758621,0.108642,0.190065,0.542556
39,HATT.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.691,0.758065,0.348148,0.477157,0.636259
88,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,LSTM.csv,0.669,0.753425,0.271605,0.399274,0.605550
111,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.658,0.733333,0.244444,0.366667,0.591970
15,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,0.681,0.712871,0.355556,0.474465,0.629038
24,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.651,0.702899,0.239506,0.357274,0.585299
30,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.691,0.695122,0.422222,0.525346,0.648086


### Continue with 3 models combinations:

In [54]:

# To generate all combinatition of pairs of models
# https://www.geeksforgeeks.org/itertools-combinations-module-python-print-possible-combinations/
from itertools import combinations

pair_iterator_3 = list(combinations(file_name, 3))

In [55]:
len(pair_iterator_3)

560

In [56]:
# for this purpose, since I need to average two files together, I feel it's easier to use pandas dataframe and use vectorized operation

df_ensemble_compare_3 = pd.DataFrame(columns = ['Model 1', 'Model 2', 'Model 3', 'Accuracy', 'Precision', 'Recall','F1','AUC'])

for files in pair_iterator_3:
    temp_pred_result = []
    
    
    # temporary hold of data from two files
    df_file_1 = pd.read_csv(files[0], 
                            names = ['Class 0', 'Class 1'], 
                            sep=" ")

    df_file_2 = pd.read_csv(files[1], 
                            names = ['Class 0', 'Class 1'], 
                            sep=" ")
    
    df_file_3 = pd.read_csv(files[2], 
                            names = ['Class 0', 'Class 1'], 
                            sep=" ")
    
    # average two files
    #df_combine = df_file_1 * 0.5 + df_file_2 * 0.5
    df_combine = (df_file_1 + df_file_2 + df_file_3) / 3

    # compute the new predicted output
    for i in df_combine.itertuples():
        temp_pred_result.append(np.argmax([i._1, i._2]))
    temp_pred_result
    
    accuracy = metrics.accuracy_score(y_test_true, temp_pred_result)
    precision = metrics.precision_score(y_test_true, temp_pred_result)
    recall = metrics.recall_score(y_test_true, temp_pred_result)
    f1 = metrics.f1_score(y_test_true, temp_pred_result)
    auc = metrics.roc_auc_score(y_test_true, temp_pred_result)
    
    
    row_to_add = {'Model 1': clean_filename(files[0]), 'Model 2': clean_filename(files[1]), 'Model 3': clean_filename(files[2]), 'Accuracy': accuracy, 'Precision':precision, 'Recall': recall,'F1': f1,'AUC': auc}
    
    df_ensemble_compare_3 = df_ensemble_compare_3.append(row_to_add, ignore_index = True)
    
df_ensemble_compare_3

,Model 1,Model 2,Model 3,Accuracy,Precision,Recall,F1,AUC
0,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,0.661,0.742647,0.249383,0.373383,0.595280
1,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.618,0.661972,0.116049,0.197479,0.537857
2,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.625,0.644231,0.165432,0.263261,0.551624
3,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,Majority_Class_prediction.csv,0.405,0.405000,1.000000,0.576512,0.500000
4,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,CNN_5yr_claims_dev_prob-Copy1.csv,0.638,0.618785,0.276543,0.382253,0.580288
...,...,...,...,...,...,...,...,...
555,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,CNN_5yr_abstract_title_dev_prob-Copy1.csv,LSTM_abst.csv,0.637,0.588235,0.345679,0.435459,0.590487
556,LSTM.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.650,0.632850,0.323457,0.428105,0.597863
557,LSTM.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,LSTM_abst.csv,0.669,0.671296,0.358025,0.466989,0.619348
558,LSTM.csv,CNN_5yr_abstract_title_dev_prob-Copy1.csv,LSTM_abst.csv,0.646,0.607595,0.355556,0.448598,0.599627


In [57]:
print("Rank by Accuracy")
df_ensemble_compare_3.sort_values(by = "Accuracy", ascending = False)

Rank by Accuracy


,Model 1,Model 2,Model 3,Accuracy,Precision,Recall,F1,AUC
22,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.712,0.785366,0.397531,0.527869,0.661791
265,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.707,0.771845,0.392593,0.520458,0.656800
114,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,LSTM.csv,0.699,0.722222,0.417284,0.528951,0.654020
214,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.698,0.703557,0.439506,0.541033,0.656728
23,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,LSTM.csv,0.697,0.705645,0.432099,0.535988,0.654705
...,...,...,...,...,...,...,...,...
431,Majority_Class_prediction.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.405,0.405000,1.000000,0.576512,0.500000
432,Majority_Class_prediction.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.405,0.405000,1.000000,0.576512,0.500000
433,Majority_Class_prediction.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,LSTM_abst.csv,0.405,0.405000,1.000000,0.576512,0.500000
434,Majority_Class_prediction.csv,LSTM.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.405,0.405000,1.000000,0.576512,0.500000


In [58]:
print("Rank by Precision")
df_ensemble_compare_3.sort_values(by = "Precision", ascending = False)

Rank by Precision


,Model 1,Model 2,Model 3,Accuracy,Precision,Recall,F1,AUC
495,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.622,0.829268,0.083951,0.152466,0.536093
243,HATT.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.651,0.825581,0.175309,0.289206,0.575049
18,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,0.646,0.822785,0.160494,0.268595,0.568482
72,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.615,0.812500,0.064198,0.118993,0.527057
163,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.621,0.809524,0.083951,0.152125,0.535253
...,...,...,...,...,...,...,...,...
399,Majority_Class_prediction.csv,CNN_5yr_claims_dev_prob-Copy1.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.405,0.405000,1.000000,0.576512,0.500000
51,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,Majority_Class_prediction.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,0.405,0.405000,1.000000,0.576512,0.500000
439,Majority_Class_prediction.csv,CNN_5yr_abstract_title_dev_prob-Copy1.csv,LSTM_abst.csv,0.405,0.405000,1.000000,0.576512,0.500000
438,Majority_Class_prediction.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,LSTM_abst.csv,0.405,0.405000,1.000000,0.576512,0.500000


### Continue with 4 models combinations:

In [59]:

# To generate all combinatition of pairs of models
# https://www.geeksforgeeks.org/itertools-combinations-module-python-print-possible-combinations/
from itertools import combinations

pair_iterator_4 = list(combinations(file_name, 4))

In [60]:
len(pair_iterator_4)

1820

In [61]:
pair_iterator_4

[('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/HATT.csv',
  'Prediction_Output/Distil-BERT_5yr_claims_dev_prob-Copy1.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/HATT.csv',
  'Prediction_Output/BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/HATT.csv',
  'Prediction_Output/Majority_Class_prediction.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/HATT.csv',
  'Prediction_Output/CNN_5yr_claims_dev_prob-Copy1.csv'),
 ('Prediction_Output/TFIDF_NaiveB

In [62]:
# for this purpose, since I need to average two files together, I feel it's easier to use pandas dataframe and use vectorized operation

df_ensemble_compare_4 = pd.DataFrame(columns = ['Model 1', 'Model 2', 'Model 3', 'Model 4', 'Accuracy', 'Precision', 'Recall','F1','AUC'])

for files in pair_iterator_4:
    temp_pred_result = []
    
    
    # temporary hold of data from two files
    df_file_1 = pd.read_csv(files[0], 
                            names = ['Class 0', 'Class 1'], 
                            sep=" ")

    df_file_2 = pd.read_csv(files[1], 
                            names = ['Class 0', 'Class 1'], 
                            sep=" ")
    
    df_file_3 = pd.read_csv(files[2], 
                            names = ['Class 0', 'Class 1'], 
                            sep=" ")
    
    df_file_4 = pd.read_csv(files[3], 
                            names = ['Class 0', 'Class 1'], 
                            sep=" ")
    
    # average two files
    #df_combine = df_file_1 * 0.5 + df_file_2 * 0.5
    df_combine = (df_file_1 + df_file_2 + df_file_3 + df_file_4) / 4

    # compute the new predicted output
    for i in df_combine.itertuples():
        temp_pred_result.append(np.argmax([i._1, i._2]))
    temp_pred_result
    
    accuracy = metrics.accuracy_score(y_test_true, temp_pred_result)
    precision = metrics.precision_score(y_test_true, temp_pred_result)
    recall = metrics.recall_score(y_test_true, temp_pred_result)
    f1 = metrics.f1_score(y_test_true, temp_pred_result)
    auc = metrics.roc_auc_score(y_test_true, temp_pred_result)
    
    
    row_to_add = {'Model 1': clean_filename(files[0]), 'Model 2': clean_filename(files[1]), 'Model 3': clean_filename(files[2]), 'Model 4': clean_filename(files[3]), 'Accuracy': accuracy, 'Precision':precision, 'Recall': recall,'F1': f1,'AUC': auc}
    
    df_ensemble_compare_4 = df_ensemble_compare_4.append(row_to_add, ignore_index = True)
    
df_ensemble_compare_4

,Model 1,Model 2,Model 3,Model 4,Accuracy,Precision,Recall,F1,AUC
0,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.656,0.756303,0.222222,0.343511,0.586741
1,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.649,0.701493,0.232099,0.348794,0.582436
2,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,Majority_Class_prediction.csv,0.405,0.405000,1.000000,0.576512,0.500000
3,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,CNN_5yr_claims_dev_prob-Copy1.csv,0.673,0.689320,0.350617,0.464812,0.621527
4,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,0.628,0.761905,0.118519,0.205128,0.546654
...,...,...,...,...,...,...,...,...,...
1815,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,LSTM.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.666,0.668246,0.348148,0.457792,0.615251
1816,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,LSTM.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,LSTM_abst.csv,0.685,0.706422,0.380247,0.494382,0.636342
1817,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,LSTM.csv,CNN_5yr_abstract_title_dev_prob-Copy1.csv,LSTM_abst.csv,0.655,0.623967,0.372840,0.466770,0.609949
1818,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,CNN_5yr_abstract_title_dev_prob-Copy1.csv,LSTM_abst.csv,0.637,0.610526,0.286420,0.389916,0.581025


In [63]:
print("Rank by Accuracy")
df_ensemble_compare_4.sort_values(by = "Accuracy", ascending = False)

Rank by Accuracy


,Model 1,Model 2,Model 3,Model 4,Accuracy,Precision,Recall,F1,AUC
523,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,LSTM.csv,0.715,0.752101,0.441975,0.556765,0.671408
110,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,LSTM.csv,0.704,0.746606,0.407407,0.527157,0.656645
159,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,LSTM.csv,0.702,0.720165,0.432099,0.540123,0.658907
98,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.701,0.791209,0.355556,0.490630,0.645845
474,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,LSTM.csv,0.701,0.747664,0.395062,0.516963,0.652153
...,...,...,...,...,...,...,...,...,...
235,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,Majority_Class_prediction.csv,CNN_5yr_claims_dev_prob-Copy1.csv,0.405,0.405000,1.000000,0.576512,0.500000
92,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,Majority_Class_prediction.csv,0.405,0.405000,1.000000,0.576512,0.500000
103,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,Majority_Class_prediction.csv,0.405,0.405000,1.000000,0.576512,0.500000
189,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,Majority_Class_prediction.csv,LSTM_abst.csv,0.405,0.405000,1.000000,0.576512,0.500000


In [64]:
print("Rank by Precision")
df_ensemble_compare_4.sort_values(by = "Precision", ascending = False)

Rank by Precision


,Model 1,Model 2,Model 3,Model 4,Accuracy,Precision,Recall,F1,AUC
136,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.665,0.830189,0.217284,0.344423,0.593516
1040,HATT.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.665,0.824074,0.219753,0.346979,0.593910
202,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.620,0.820513,0.079012,0.144144,0.533624
393,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,LSTM.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.641,0.819444,0.145679,0.247379,0.561915
138,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.637,0.818182,0.133333,0.229299,0.556583
...,...,...,...,...,...,...,...,...,...
1179,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,Majority_Class_prediction.csv,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.405,0.405000,1.000000,0.576512,0.500000
1180,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,Majority_Class_prediction.csv,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,LSTM.csv,0.405,0.405000,1.000000,0.576512,0.500000
1181,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,Majority_Class_prediction.csv,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.405,0.405000,1.000000,0.576512,0.500000
1182,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,Majority_Class_prediction.csv,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.405,0.405000,1.000000,0.576512,0.500000


### Now, let's merge the Ensemble models with the original single models and rank them altogether for comparison purpose

In [65]:
# Create a copy to preserve the original table
summary_df_copy = summary_df.copy()

# Rename the column name to align both tables
summary_df_copy = summary_df_copy.rename(columns={"Model": "Model 1"})

# Concatenate both tables (this merge based on the column header)
df_all = pd.concat([df_ensemble_compare_4, df_ensemble_compare_3, df_ensemble_compare_2 , summary_df_copy])
df_all

,Model 1,Model 2,Model 3,Model 4,Accuracy,Precision,Recall,F1,AUC
0,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.656,0.756303,0.222222,0.343511,0.586741
1,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.649,0.701493,0.232099,0.348794,0.582436
2,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,Majority_Class_prediction.csv,0.405,0.405000,1.000000,0.576512,0.500000
3,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,CNN_5yr_claims_dev_prob-Copy1.csv,0.673,0.689320,0.350617,0.464812,0.621527
4,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,0.628,0.761905,0.118519,0.205128,0.546654
...,...,...,...,...,...,...,...,...,...
11,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,NaN,NaN,NaN,0.689,0.636000,0.543000,0.586000,0.666000
12,LSTM.csv,NaN,NaN,NaN,0.681,0.623000,0.538000,0.577000,0.658000
13,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,NaN,NaN,NaN,0.618,0.632000,0.136000,0.224000,0.541000
14,CNN_5yr_abstract_title_dev_prob-Copy1.csv,NaN,NaN,NaN,0.589,0.489000,0.316000,0.384000,0.545000


In [66]:
# Now we can sort/rank them again
print("Rank by Accuracy")
df_all.sort_values(by = "Accuracy", ascending = False)


Rank by Accuracy


,Model 1,Model 2,Model 3,Model 4,Accuracy,Precision,Recall,F1,AUC
523,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,LSTM.csv,0.715,0.752101,0.441975,0.556765,0.671408
22,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,NaN,0.712,0.785366,0.397531,0.527869,0.661791
265,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,NaN,0.707,0.771845,0.392593,0.520458,0.656800
110,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,LSTM.csv,0.704,0.746606,0.407407,0.527157,0.656645
159,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,LSTM.csv,0.702,0.720165,0.432099,0.540123,0.658907
...,...,...,...,...,...,...,...,...,...
959,HATT.csv,Majority_Class_prediction.csv,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.405,0.405000,1.000000,0.576512,0.500000
958,HATT.csv,Majority_Class_prediction.csv,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,HATT_abst.csv,0.405,0.405000,1.000000,0.576512,0.500000
957,HATT.csv,Majority_Class_prediction.csv,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,0.405,0.405000,1.000000,0.576512,0.500000
956,HATT.csv,Majority_Class_prediction.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,LSTM_abst.csv,0.405,0.405000,1.000000,0.576512,0.500000


In [67]:
print("Rank by Precision")
df_all.sort_values(by = "Precision", ascending = False)


Rank by Precision


,Model 1,Model 2,Model 3,Model 4,Accuracy,Precision,Recall,F1,AUC
136,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.665,0.830189,0.217284,0.344423,0.593516
495,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,NaN,0.622,0.829268,0.083951,0.152466,0.536093
243,HATT.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,NaN,0.651,0.825581,0.175309,0.289206,0.575049
1040,HATT.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.665,0.824074,0.219753,0.346979,0.593910
18,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,NaN,0.646,0.822785,0.160494,0.268595,0.568482
...,...,...,...,...,...,...,...,...,...
208,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,Majority_Class_prediction.csv,NaN,0.405,0.405000,1.000000,0.576512,0.500000
966,HATT.csv,Majority_Class_prediction.csv,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,LSTM.csv,0.405,0.405000,1.000000,0.576512,0.500000
965,HATT.csv,Majority_Class_prediction.csv,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.405,0.405000,1.000000,0.576512,0.500000
964,HATT.csv,Majority_Class_prediction.csv,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,HATT_abst.csv,0.405,0.405000,1.000000,0.576512,0.500000


In [68]:
print("Rank by Recall")
df_all.sort_values(by = "Recall", ascending = False)


Rank by Recall


,Model 1,Model 2,Model 3,Model 4,Accuracy,Precision,Recall,F1,AUC
1528,Majority_Class_prediction.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.405,0.405000,1.000000,0.576512,0.500000
396,Majority_Class_prediction.csv,CNN_5yr_claims_dev_prob-Copy1.csv,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,NaN,0.405,0.405000,1.000000,0.576512,0.500000
1338,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,Majority_Class_prediction.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,LSTM.csv,0.405,0.405000,1.000000,0.576512,0.500000
1337,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,Majority_Class_prediction.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.405,0.405000,1.000000,0.576512,0.500000
1336,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,Majority_Class_prediction.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,HATT_abst.csv,0.405,0.405000,1.000000,0.576512,0.500000
...,...,...,...,...,...,...,...,...,...
5,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,NaN,0.603,0.653846,0.041975,0.078886,0.513425
89,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,NaN,NaN,0.599,0.611111,0.027160,0.052009,0.507698
74,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,NaN,0.597,0.562500,0.022222,0.042755,0.505229
6,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,NaN,NaN,0.597,0.571429,0.019753,0.038186,0.504835


In [69]:
print("Rank by F1")
df_all.sort_values(by = "F1", ascending = False)


Rank by F1


,Model 1,Model 2,Model 3,Model 4,Accuracy,Precision,Recall,F1,AUC
11,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,NaN,NaN,NaN,0.689,0.636000,0.543000,0.586000,0.666000
255,HATT.csv,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,LSTM.csv,NaN,0.695,0.657233,0.516049,0.578147,0.666428
38,HATT.csv,LSTM.csv,NaN,NaN,0.690,0.644377,0.523457,0.577657,0.663409
12,LSTM.csv,NaN,NaN,NaN,0.681,0.623000,0.538000,0.577000,0.658000
1497,Majority_Class_prediction.csv,CNN_5yr_claims_dev_prob-Copy1.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,LSTM_abst.csv,0.405,0.405000,1.000000,0.576512,0.500000
...,...,...,...,...,...,...,...,...,...
30,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,NaN,0.602,0.629630,0.041975,0.078704,0.512584
89,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,NaN,NaN,0.599,0.611111,0.027160,0.052009,0.507698
74,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,NaN,0.597,0.562500,0.022222,0.042755,0.505229
6,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_claims_last_510_words_dev_prob.csv,NaN,NaN,0.597,0.571429,0.019753,0.038186,0.504835


In [70]:
print("Rank by AUC")
df_all.sort_values(by = "AUC", ascending = False)


Rank by AUC


,Model 1,Model 2,Model 3,Model 4,Accuracy,Precision,Recall,F1,AUC
523,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,LSTM.csv,0.715,0.752101,0.441975,0.556765,0.671408
255,HATT.csv,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,LSTM.csv,NaN,0.695,0.657233,0.516049,0.578147,0.666428
11,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,NaN,NaN,NaN,0.689,0.636000,0.543000,0.586000,0.666000
38,HATT.csv,LSTM.csv,NaN,NaN,0.690,0.644377,0.523457,0.577657,0.663409
1075,HATT.csv,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,LSTM.csv,0.695,0.665563,0.496296,0.568600,0.663274
...,...,...,...,...,...,...,...,...,...
119,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,Majority_Class_prediction.csv,NaN,0.405,0.405000,1.000000,0.576512,0.500000
963,HATT.csv,Majority_Class_prediction.csv,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,LSTM_abst.csv,0.405,0.405000,1.000000,0.576512,0.500000
962,HATT.csv,Majority_Class_prediction.csv,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.405,0.405000,1.000000,0.576512,0.500000
961,HATT.csv,Majority_Class_prediction.csv,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.405,0.405000,1.000000,0.576512,0.500000
